In [8]:
import numpy as np
import time

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1.0 - np.tanh(x)**2

def logistic(x):
    return 1/(1 + np.exp(-x))

def logistic_derivative(x):
    return logistic(x)*(1-logistic(x))

In [9]:
class NeuralNetwork:
    def __init__(self, layers, activation='tanh'):
        """
        :param layers: A list containing the number of units in each layer.
        Should be at least two values
        :param activation: The activation function to be used. Can be
        "logistic" or "tanh"
        """
        if activation == 'logistic':
            self.activation = logistic
            self.activation_deriv = logistic_derivative
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_deriv = tanh_deriv

        self.weights = []
        for i in range(1, len(layers) - 1):
            self.weights.append((2*np.random.random((layers[i - 1] + 1, layers[i]
                                ))-1)*0.25)
        self.weights.append((2*np.random.random((layers[i] + 1, layers[i +
                            1]))-1)*0.25)
        
    def fit(self, X, y, learning_rate=0.2, epochs=10000):
        X = np.atleast_2d(X)
        temp = np.ones([X.shape[0], X.shape[1]+1])
        temp[:, 0:-1] = X  # adding the bias unit to the input layer
        X = temp
        y = np.array(y)

        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                hidden_inputs = np.ones([self.weights[l].shape[1] + 1])
                hidden_inputs[0:-1] = self.activation(np.dot(a[l], self.weights[l]))
                a.append(hidden_inputs)
            error = y[i] - a[-1][:-1]
            deltas = [error * self.activation_deriv(a[-1][:-1])]
            l = len(a) - 2

            # The last layer before the output is handled separately because of
            # the lack of bias node in output
            deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_deriv(a[l]))

            for l in range(len(a) -3, 0, -1): # we need to begin at the second to last layer
                deltas.append(deltas[-1][:-1].dot(self.weights[l].T)*self.activation_deriv(a[l]))

            deltas.reverse()
            for i in range(len(self.weights)-1):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta[:,:-1])
            # Handle last layer separately because it doesn't have a bias unit
            i+=1
            layer = np.atleast_2d(a[i])
            delta = np.atleast_2d(deltas[i])
            self.weights[i] += learning_rate * layer.T.dot(delta)
            
    def predict(self, x):
        a = np.array(x)
        for l in range(0, len(self.weights)):
            temp = np.ones(a.shape[0]+1)
            temp[0:-1] = a
            a = self.activation(np.dot(temp, self.weights[l]))
        return a

In [10]:
nn = NeuralNetwork([2,2,1], 'tanh')
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
start_time = time.time()
nn.fit(X, y)
print("training time : " + str( time.time() - start_time))
for i in [[0, 0], [0, 1], [1, 0], [1,1]]:
    print(i,nn.predict(i))

training time : 0.9853711128234863
[0, 0] [0.45824737]
[0, 1] [0.46261759]
[1, 0] [0.46062954]
[1, 1] [0.46500399]


In [16]:
import numpy as np
from sklearn.cross_validation import train_test_split 
from sklearn.datasets import load_digits
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer


digits = load_digits()
X = digits.data
y = digits.target
X -= X.min() # normalize the values to bring them into the range 0-1
X /= X.max()
print(X.shape)
nn = NeuralNetwork([64,100,50,40,10],'tanh')
X_train, X_test, y_train, y_test = train_test_split(X, y)
labels_train = LabelBinarizer().fit_transform(y_train)
labels_test = LabelBinarizer().fit_transform(y_test)

start_time = time.time()
nn.fit(X_train,labels_train,epochs=30000)
print("training time : " + str( time.time() - start_time))
predictions = []
for i in range(X_test.shape[0]):
    o = nn.predict(X_test[i] )
    predictions.append(np.argmax(o))
print (confusion_matrix(y_test,predictions))
print (classification_report(y_test,predictions))

(1797, 64)
training time : 10.833237171173096
[[34  0  0  0  0  0  0  0  0  0]
 [ 1 46  0  0  0  0  0  0  0  0]
 [ 0  5 38  0  0  0  0  0  0  0]
 [ 0  0  0 40  0  0  0  0  1  0]
 [ 0  4  0  0 42  0  0  1  0  0]
 [ 0  0  0  0  0 44  0  0  0  1]
 [ 4  2  0  0  0  0 49  0  0  0]
 [ 0  0  0  0  0  0  0 54  0  0]
 [ 1  3  0  1  0  0  0  1 31  3]
 [ 0  0  0  0  0  0  0  2  0 42]]
             precision    recall  f1-score   support

          0       0.85      1.00      0.92        34
          1       0.77      0.98      0.86        47
          2       1.00      0.88      0.94        43
          3       0.98      0.98      0.98        41
          4       1.00      0.89      0.94        47
          5       1.00      0.98      0.99        45
          6       1.00      0.89      0.94        55
          7       0.93      1.00      0.96        54
          8       0.97      0.78      0.86        40
          9       0.91      0.95      0.93        44

avg / total       0.94      0.93      